In [45]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense,Dropout, LSTM, RepeatVector, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error
from keras.utils import plot_model

In [2]:
df = pd.read_csv('outputFiles/PCC1/cec.csv', index_col=0, engine='c')

In [3]:
df_ = pd.read_csv('outputFiles/PCC3/cec.csv', index_col=0, engine='c')

In [4]:
outliers = np.array(df_)

In [5]:
outliers

array([[2.30591105, 2.35490158, 2.37487914, ..., 1.32834015, 1.3276394 ,
        1.3270798 ],
       [2.15990168, 2.20330317, 2.20656454, ..., 1.40237046, 1.40162838,
        1.40181444],
       [2.25442973, 2.24240072, 2.19712424, ..., 1.17695509, 1.1757542 ,
        1.17475592],
       ...,
       [1.88627901, 1.90838998, 1.8863147 , ..., 1.17715932, 1.17688827,
        1.17794835],
       [2.36346882, 2.33737526, 2.35798084, ..., 1.309042  , 1.30812407,
        1.30789195],
       [2.34791306, 2.21431707, 2.13118097, ..., 1.28921092, 1.28926904,
        1.29004676]])

In [18]:
X = np.array(df)

In [20]:
np.shape(X)

(14129, 1728)

In [21]:
def normalize(data):
    min_val = np.min(data)
    max_val = np.max(data)
    normalized_data = 2 * (data - min_val) / (max_val - min_val) - 1
    return normalized_data

In [22]:
def denormalize(normalized_data, original_min, original_max):
    denormalized_data = (normalized_data + 1) * (original_max - original_min) / 2 + original_min
    return denormalized_data

In [23]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      Dense(32, activation="relu"),
      Dense(16, activation="relu"),
      Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      Dense(16, activation="relu"),
      Dense(32, activation="relu"),
      Dense(1728, activation="tanh")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [24]:
X_spc = []

In [25]:
for i in range(len(X)):
    X_spc.append(normalize(X[i]))

In [31]:
X_spc = np.array(X_spc)

In [33]:
X_spc.shape

(14129, 1728)

In [34]:
autoencoder = AnomalyDetector()

In [35]:
autoencoder.compile(optimizer='adam', loss='mse')

In [36]:
history = autoencoder.fit(X_spc, X_spc, 
          epochs=100, 
          batch_size=32,
          validation_data=(X_spc, X_spc),
          shuffle=True)

Epoch 1/20
442/442 [==============================] - 4s 8ms/step - loss: 0.0304 - val_loss: 0.0155
Epoch 2/20
442/442 [==============================] - 3s 8ms/step - loss: 0.0141 - val_loss: 0.0075
Epoch 3/20
442/442 [==============================] - 3s 8ms/step - loss: 0.0050 - val_loss: 0.0043
Epoch 4/20
442/442 [==============================] - 3s 8ms/step - loss: 0.0041 - val_loss: 0.0040
Epoch 5/20
442/442 [==============================] - 3s 8ms/step - loss: 0.0040 - val_loss: 0.0039
Epoch 6/20
442/442 [==============================] - 3s 8ms/step - loss: 0.0038 - val_loss: 0.0038
Epoch 7/20
442/442 [==============================] - 3s 8ms/step - loss: 0.0036 - val_loss: 0.0033
Epoch 8/20
442/442 [==============================] - 3s 8ms/step - loss: 0.0030 - val_loss: 0.0030
Epoch 9/20
442/442 [==============================] - 3s 8ms/step - loss: 0.0029 - val_loss: 0.0029
Epoch 10/20
442/442 [==============================] - 4s 8ms/step - loss: 0.0028 - val_loss: 0.0028

In [37]:
autoencoder.summary()

Model: "anomaly_detector_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 8)                 55992     
                                                                 
 sequential_3 (Sequential)   (None, 1728)              57712     
                                                                 
Total params: 113,704
Trainable params: 113,704
Non-trainable params: 0
_________________________________________________________________


In [38]:
orig_max

NameError: name 'orig_max' is not defined

In [39]:
pred = autoencoder.predict(X_spc)

442/442 [==============================] - 1s 2ms/step


In [42]:
predictions = []

In [43]:
for reconstructed, orig_spc in zip(pred, X):
    min = (np.min(orig_spc))
    max = (np.max(orig_spc))
    predictions.append(denormalize(reconstructed, min, max))

In [48]:
mae = []
mse = []

In [49]:
for orig, reconstructed in zip(X, predictions):
    mae.append(mean_absolute_error(orig, reconstructed))
    mse.append(mean_squared_error(orig, reconstructed))

In [51]:
mae

[0.019072183653791912,
 0.019414086027261673,
 0.014970417315006812,
 0.012176047361718685,
 0.017462707336037428,
 0.014681845985739416,
 0.0368905690314501,
 0.018431541341685644,
 0.02973630638322302,
 0.025699096713328488,
 0.03207629339729505,
 0.03744578764418635,
 0.026894490170013902,
 0.02660887180109217,
 0.03350326950562245,
 0.014982017381080738,
 0.017457419592788098,
 0.023646048748258788,
 0.020319904967336287,
 0.018562786580097163,
 0.029138467335945113,
 0.02102329872264229,
 0.03312721679787564,
 0.016704152680818263,
 0.020891706028018684,
 0.017920275535906072,
 0.01517477368677688,
 0.015461254533554892,
 0.011113057628718939,
 0.019851458670890828,
 0.010412724561382616,
 0.011189646094074135,
 0.012837571999454662,
 0.019727822870587528,
 0.01285743702965487,
 0.014878267767125612,
 0.01848726081044169,
 0.018593791856349417,
 0.01460492336309892,
 0.01701390110450466,
 0.016446511028345462,
 0.021839642399717214,
 0.016908117834129186,
 0.01787897971843198,
 0.

In [53]:
outlier_spc = []

In [54]:
for spc in outliers:
    outlier_spc.append(normalize(spc))

In [56]:
outlier_spc = np.array(outlier_spc)

In [57]:
outlier_pred = autoencoder.predict(outlier_spc)

2/2 [==============================] - 0s 8ms/step


In [61]:
outlier_predictions = []

In [62]:
for reconstructed, orig_spc in zip(outlier_pred, outliers):
    min = (np.min(orig_spc))
    max = (np.max(orig_spc))
    outlier_predictions.append(denormalize(reconstructed, min, max))

In [64]:
outlier_mae = []
outlier_mse = []

In [65]:
for orig, reconstructed in zip(outliers, outlier_predictions):
    outlier_mae.append(mean_absolute_error(orig, reconstructed))
    outlier_mse.append(mean_squared_error(orig, reconstructed))

In [67]:
outlier_mae

[0.015465776750906537,
 0.017192145997969692,
 0.0300400884905724,
 0.11437339356144925,
 0.02755190658426583,
 0.02869274538309789,
 0.027267981304864942,
 0.06060762750487593,
 0.021173003873048164,
 0.029587312652362415,
 0.05031516737594724,
 0.03531324917477384,
 0.03798079866502887,
 0.020589452217139325,
 0.02858928732375594,
 0.030981240949352142,
 0.03434473691646832,
 0.04843446254698729,
 0.03179921333361753,
 0.037666435583014926,
 0.037920181059813984,
 0.03812815788044638,
 0.03723547137269497,
 0.033802385482530055,
 0.023999497452514396,
 0.024412957404704866,
 0.027599338276923655,
 0.031029818205197107,
 0.023805969363132992,
 0.05795448520646098,
 0.021269965418368654,
 0.017111288829537313,
 0.039210508054705905,
 0.02437994070814724,
 0.02298468685535696,
 0.02506718445897861,
 0.05511945980807029,
 0.020972332052861392,
 0.028092834455876976,
 0.0252884809232435,
 0.044120162943673484,
 0.028976650256239803,
 0.03079874565121577,
 0.027615638966673475,
 0.02379491

In [100]:
(len([i for i in mae if i < min]) / len(X)) * 100

6.87946776134192

In [126]:
modelling_spc = []

In [127]:
for spc, err in zip(X, mae):
    if(err < np.min(outlier_mae)):
        modelling_spc.append(normalize(spc))

In [128]:
modelling_spc = np.array(modelling_spc)

In [129]:
model = AnomalyDetector()

In [130]:
model.compile(optimizer='adam', loss='mse')

In [131]:
history = model.fit(modelling_spc, modelling_spc, 
          epochs=100, 
          batch_size=32,
          validation_data=(modelling_spc, modelling_spc),
          shuffle=True)

Epoch 1/100
31/31 [==============================] - 4s 28ms/step - loss: 0.1452 - val_loss: 0.0307
Epoch 2/100
31/31 [==============================] - 0s 11ms/step - loss: 0.0250 - val_loss: 0.0223
Epoch 3/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0203 - val_loss: 0.0163
Epoch 4/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0134 - val_loss: 0.0116
Epoch 5/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0113 - val_loss: 0.0110
Epoch 6/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0110 - val_loss: 0.0108
Epoch 7/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0108 - val_loss: 0.0107
Epoch 8/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0107 - val_loss: 0.0105
Epoch 9/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0104 - val_loss: 0.0103
Epoch 10/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0101 - val_loss: 0.0098
Epoch 1

Epoch 83/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0017 - val_loss: 0.0016
Epoch 84/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 85/100
31/31 [==============================] - 0s 9ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 86/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0017 - val_loss: 0.0016
Epoch 87/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 88/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0017 - val_loss: 0.0016
Epoch 89/100
31/31 [==============================] - 0s 9ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 90/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 91/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0016 - val_loss: 0.0016
Epoch 92/100
31/31 [==============================] - 0s 8ms/step - loss: 0.0016 - val_loss: 0.0016


In [132]:
final_preds = model.predict(modelling_spc)

31/31 [==============================] - 0s 2ms/step


In [134]:
rec_spc = []

In [135]:
for reconstructed, orig_spc in zip(final_preds, modelling_spc):
    min = (np.min(orig_spc))
    max = (np.max(orig_spc))
    rec_spc.append(denormalize(reconstructed, min, max))

In [136]:
rec_spc

[array([ 0.25693846,  0.23780036,  0.21972239, ..., -0.99843746,
        -0.9988283 , -0.9988219 ], dtype=float32),
 array([ 0.25626743,  0.2363491 ,  0.21682394, ..., -0.99840605,
        -0.9988213 , -0.9988145 ], dtype=float32),
 array([ 0.13860273,  0.11833382,  0.08779991, ..., -0.9997348 ,
        -0.99981743, -0.9998319 ], dtype=float32),
 array([ 0.63469696,  0.6325226 ,  0.62688303, ..., -0.99776137,
        -0.9984333 , -0.99820966], dtype=float32),
 array([ 0.2554512 ,  0.23367035,  0.21060848, ..., -0.99835384,
        -0.99882394, -0.9988158 ], dtype=float32),
 array([ 0.49925876,  0.48729098,  0.4726591 , ..., -0.9982118 ,
        -0.99876696, -0.9986134 ], dtype=float32),
 array([ 0.38755083,  0.36806095,  0.3491075 , ..., -0.9985286 ,
        -0.99875396, -0.99898165], dtype=float32),
 array([ 0.486403  ,  0.47335815,  0.46221805, ..., -0.9982468 ,
        -0.99876904, -0.998572  ], dtype=float32),
 array([ 0.4094708 ,  0.39348292,  0.37757635, ..., -0.9977497 ,
       